# Notebook 4: Assessing Potential Impact of Energy News

This notebook demonstrates the experimental impact assessment feature using the `analysis` service.

**Objective:** Use a simple LLM chain (via Langchain and a Hugging Face model like GPT-2) to provide a qualitative assessment of the potential impact of a news article on a specified context (e.g., the energy market, oil prices).

**Disclaimer:** The impact assessment provided by the current basic model (`gpt2`) is experimental and likely simplistic. A more sophisticated assessment would require a larger, more capable LLM and potentially more complex prompting or a dedicated reasoning agent.

## 1. Setup

Import libraries, add the project root to the path, and load the analysis service. This will load the LLM pipeline specified in `analysis.py`.

In [ ]:
import osimport sysimport jsonimport logging# Setup logginglogging.basicConfig(level=logging.INFO)logger = logging.getLogger(__name__)# Add the project root to the Python pathproject_root = os.path.abspath(os.path.join(os.getcwd(), '..'))if project_root not in sys.path:    sys.path.append(project_root)# Import the analysis servicetry:    from api.services import analysis    import importlib    importlib.reload(analysis) # Ensure latest code and models are loaded    models_loaded = analysis.impact_llm is not None    if models_loaded:        logger.info('Impact assessment LLM pipeline loaded successfully.')    else:        logger.warning('Impact assessment LLM pipeline failed to load. Check logs in analysis.py.')except Exception as e:    logger.error(f'Failed to import or load analysis service: {e}', exc_info=True)    models_loaded = False

## 2. Load Summaries (or Original Articles)

We can use either the full article content or the summaries generated in the previous notebook as input for the impact assessment. Summaries are generally better as they provide concise information to the LLM. We'll load the summaries first.

In [ ]:
summaries = []summaries_file = os.path.join("data", "summary_results.json")articles = [] # Also load original articles for fallbackarticles_file = os.path.join("data", "fetched_articles.json")if os.path.exists(summaries_file):    try:        with open(summaries_file, 'r') as f:            summaries = json.load(f)        print(f"Loaded {len(summaries)} summaries from {summaries_file}")    except Exception as e:        print(f"Error loading summaries from {summaries_file}: {e}")else:    print(f"Warning: {summaries_file} not found. Will try to use original articles.")if not summaries:    if os.path.exists(articles_file):        try:            with open(articles_file, 'r') as f:                articles = json.load(f)            print(f"Loaded {len(articles)} original articles from {articles_file}")        except Exception as e:            print(f"Error loading articles from {articles_file}: {e}")    else:       print(f"Error: Neither summaries nor original articles found. Please run Notebook 1 and 3 first.")

## 3. Perform Impact Assessment

Iterate through the summaries (or articles) and apply the `assess_impact` function. We need to provide a context for the assessment (e.g., 'energy market', 'oil prices', 'renewable energy stocks').

In [ ]:
if models_loaded and (summaries or articles):    print("Performing impact assessment...")    results = []    # Determine the input source (summaries preferred)    input_data = summaries if summaries else articles    context = "global energy market" # Define a general context        for i, item in enumerate(input_data):        title = item.get('title', 'No Title')        # Get text: summary if available, else content/description from original article        text_to_assess = None        if 'summary' in item and item['summary']:            text_to_assess = item['summary']        elif 'content' in item and item['content']:             text_to_assess = item['content']        elif 'description' in item and item['description']:             text_to_assess = item['description']        print(f"\nAssessing impact for article {i+1}: {title}")        print(f"Context: {context}")                if text_to_assess:            impact_assessment = analysis.assess_impact(text_to_assess, context=context)            if impact_assessment:                print(f"  Potential Impact: {impact_assessment}")                results.append({                    'title': title,                    'url': item.get('url'),                    'context': context,                    'assessment': impact_assessment                })            else:                print("  Impact assessment failed for this article.")                results.append({                    'title': title,                    'url': item.get('url'),                    'error': 'Impact assessment failed'                })        else:            print("  Skipping assessment: No summary or content available.")            results.append({                'title': title,                'url': item.get('url'),                'error': 'No text available for assessment'            })                # Save results    results_file = os.path.join("data", "impact_assessment_results.json")    with open(results_file, 'w') as f:        json.dump(results, f, indent=2)    print(f"\nSaved impact assessment results to {results_file}")            elif not models_loaded:    print("Skipping impact assessment as the LLM pipeline failed to load.")elif not summaries and not articles:    print("Skipping impact assessment as no summaries or articles were loaded.")

## 4. Conclusion

This notebook demonstrated the experimental impact assessment using a basic LLM chain provided by the `analysis.py` service. The qualitative assessments for the specified context have been saved to `notebooks/data/impact_assessment_results.json`. Remember that the quality of this assessment is highly dependent on the underlying LLM used.